In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# 1. LOAD DATA
df_sp = pd.read_csv('keberadaan_simpan_pinjam.csv')
df_sinyal = pd.read_csv('sinyal_telepon.csv')
df_peri = pd.read_csv('teknologi_perikanan.csv')
df_pete = pd.read_csv('teknologi_peternakan.csv')
df_kes = pd.read_csv('layanan_kesehatan.csv')

In [3]:
# Fungsi untuk membersihkan kolom kode agar bisa digabung (Merge Key)
def clean_kode(series):
    return series.astype(str).str.strip().str.replace('.0', '', regex=False)

# Terapkan ke semua dataframe yang punya kode desa
for df in [df_sp, df_sinyal, df_peri, df_pete]:
    df['tahun'] = df['tahun'].fillna(0).astype(int)
    # Bersihkan kode desa
    df['kemendagri_kode_desa_kelurahan'] = clean_kode(df['kemendagri_kode_desa_kelurahan'])

In [4]:
# 2. FILTER TAHUN 2023 & PREPROCESSING FITUR
# Simpan Pinjam (Target: skor_simpan_pinjam)
df_sp_23 = df_sp[df_sp['tahun'] == 2023].copy()
df_sp_23['skor_simpan_pinjam'] = df_sp_23['keberadaan_bumdesa_keuangan_program_usaha_ekonomi_desa'].apply(
    lambda x: 1 if x == 'ADA' else 0
)
# Ambil kolom kunci dan fitur saja
df_base = df_sp_23[['kemendagri_kode_desa_kelurahan', 'bps_kode_kabupaten_kota', 'skor_simpan_pinjam']]

In [5]:
# Sinyal (Target: skor_sinyal)
df_sinyal_23 = df_sinyal[df_sinyal['tahun'] == 2023].copy()
map_sinyal = {'SINYAL KUAT': 3, 'SINYAL LEMAH': 2, 'TIDAK ADA SINYAL': 1}
df_sinyal_23['skor_sinyal'] = df_sinyal_23['status_sinyal_telepon_seluler'].map(map_sinyal).fillna(1)
df_sinyal_clean = df_sinyal_23[['kemendagri_kode_desa_kelurahan', 'skor_sinyal']]

In [6]:
# Alat Perikanan & Peternakan (Target: jumlah alat)
df_peri_clean = df_peri[df_peri['tahun'] == 2023][['kemendagri_kode_desa_kelurahan', 'jumlah_alat_teknologi_tepat_guna_perikanan']]
df_pete_clean = df_pete[df_pete['tahun'] == 2023][['kemendagri_kode_desa_kelurahan', 'jumlah_alat_teknologi_tepat_guna_peternakan']]

In [7]:
# Kesehatan (Agregasi per Kabupaten)
df_kes_23 = df_kes[df_kes['tahun'] == 2023].copy()
map_jarak = {'0-1000 M': 5, '1001-2000 M': 4, '2001-3000 M': 3, '3001-4000 M': 2, '>4000 M': 1}
df_kes_23['bobot'] = df_kes_23['jarak'].map(map_jarak)
df_kes_23['total_skor'] = df_kes_23['jumlah_desa'] * df_kes_23['bobot']

In [8]:
# Hitung rata-rata skor per kabupaten
kab_kesehatan = df_kes_23.groupby('kode_kabupaten_kota').apply(
    lambda x: x['total_skor'].sum() / x['jumlah_desa'].sum()
).reset_index(name='skor_kesehatan')

/tmp/ipython-input-3810721243.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  kab_kesehatan = df_kes_23.groupby('kode_kabupaten_kota').apply(


In [9]:
# Bersihkan kode kabupaten agar bisa di-merge (float ke int ke str)
kab_kesehatan['kode_kabupaten_kota'] = kab_kesehatan['kode_kabupaten_kota'].astype(int).astype(str)
df_base['bps_kode_kabupaten_kota'] = df_base['bps_kode_kabupaten_kota'].fillna(0).astype(int).astype(str)

/tmp/ipython-input-1079005555.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base['bps_kode_kabupaten_kota'] = df_base['bps_kode_kabupaten_kota'].fillna(0).astype(int).astype(str)


In [10]:
# 3. MERGING (PENGGABUNGAN DATA)
# Gunakan 'how=left' agar data tidak hilang jika ada yang bolong, lalu isi dengan 0
df_final = df_base.merge(df_sinyal_clean, on='kemendagri_kode_desa_kelurahan', how='left')
df_final = df_final.merge(df_peri_clean, on='kemendagri_kode_desa_kelurahan', how='left')
df_final = df_final.merge(df_pete_clean, on='kemendagri_kode_desa_kelurahan', how='left')
df_final = df_final.merge(kab_kesehatan, left_on='bps_kode_kabupaten_kota', right_on='kode_kabupaten_kota', how='left')

In [11]:
# Isi nilai NaN (Kosong) dengan 0 atau rata-rata
df_final = df_final.fillna(0)

print(f"Jumlah Data Desa Berhasil Diproses: {len(df_final)} baris")

Jumlah Data Desa Berhasil Diproses: 5311 baris


In [12]:
# 4. MODELING
if len(df_final) > 0:
    features = ['skor_simpan_pinjam', 'skor_sinyal',
                'jumlah_alat_teknologi_tepat_guna_perikanan',
                'jumlah_alat_teknologi_tepat_guna_peternakan',
                'skor_kesehatan']

    X = df_final[features]

    # Scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # K-Means
    kmeans = KMeans(n_clusters=3, random_state=42)
    df_final['cluster'] = kmeans.fit_predict(X_scaled)

    print("\n--- Rata-rata Fitur per Cluster ---")
    print(df_final.groupby('cluster')[features].mean())

    # Save hasil
    df_final.to_csv('hasil_clustering_desa_fixed.csv', index=False)
    print("\nFile 'hasil_clustering_desa_fixed.csv' berhasil disimpan!")
else:
    print("Error: Data masih kosong. Cek kembali nama kolom atau isi CSV.")


--- Rata-rata Fitur per Cluster ---
         skor_simpan_pinjam  skor_sinyal  \
cluster                                    
0                       0.0          0.0   
1                       0.0          0.0   
2                       1.0          0.0   

         jumlah_alat_teknologi_tepat_guna_perikanan  \
cluster                                               
0                                               0.0   
1                                               0.0   
2                                               0.0   

         jumlah_alat_teknologi_tepat_guna_peternakan  skor_kesehatan  
cluster                                                               
0                                                0.0        4.605844  
1                                                0.0        4.866669  
2                                                0.0        4.732166  

File 'hasil_clustering_desa_fixed.csv' berhasil disimpan!
